# Lab 10 - Module 3: Analysis and Visualization

**Time:** ~15-20 minutes

In this module, you'll visualize the patterns in your data to answer the core question:

> **Does AI confidence match actual accuracy?**

You'll create three visualizations:
1. **Confusion Matrix Heatmap** - How often confidence matched performance
2. **Error Rate Bar Chart** - Which prompt categories had the most errors
3. **Overconfidence Examples** - Specific cases where AI was confident but wrong

These visualizations will reveal patterns that might be invisible when looking at individual prompts.

## Setup: Import Libraries and Load Data

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, Markdown

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries loaded successfully!")

## Generate Prompts for Your Group

Use the **same group code** from Module 0 to regenerate your prompts.

In [ ]:
def generate_group_prompts(group_code, num_prompts=8):
    """Generate deterministic prompts for a group."""
    np.random.seed(group_code)

    # Prompt pools (10 per category)
    pools = {
        'factual_recall': [
            "List the 5 largest freshwater lakes in Africa by volume.",
            "What was the population of Iceland in 1950?",
            "Name all countries that border Mongolia.",
            "Who was the Prime Minister of Canada in 1985?",
            "What is the capital city of Bhutan?",
            "List all chemical elements discovered before 1800.",
            "What year was the first transatlantic telegraph cable completed?",
            "Name the five longest rivers in South America by length.",
            "Who won the Nobel Prize in Literature in 1962?",
            "What is the elevation of Mount Kilimanjaro in meters?"
        ],
        'reasoning_chain': [
            "If it takes 5 machines 5 minutes to make 5 widgets, how long does it take 100 machines to make 100 widgets?",
            "A bat and ball cost $1.10 total. The bat costs $1.00 more than the ball. How much does the ball cost?",
            "If you overtake the person in second place in a race, what place are you in?",
            "A farmer has 17 sheep. All but 9 die. How many sheep remain?",
            "You have a 3-gallon jug and a 5-gallon jug. How can you measure exactly 4 gallons?",
            "If a plane crashes on the border of the US and Canada, where do you bury the survivors?",
            "A rope ladder hangs over the side of a ship. The rungs are 1 foot apart. At low tide, 10 rungs are below water. When the tide rises 5 feet, how many rungs are below water?",
            "Mary's father has 5 daughters: Nana, Nene, Nini, Nono. What is the 5th daughter's name?",
            "A book costs $10 plus half its price. What's the total price?",
            "If a doctor gives you 3 pills and says to take one every half hour, how long until you've taken all of them?"
        ],
        'citation_request': [
            "List 3 peer-reviewed studies published since 2020 about standing desks and productivity, including authors and journal names.",
            "Provide 2 court cases from 2023 about cryptocurrency regulation in Texas, including case numbers.",
            "Cite 3 research papers on the health effects of intermittent fasting published in Nature or Science since 2022.",
            "List some quotes from Shakespeare plays that mention the word 'computer'.",
            "Provide citations for 3 studies showing that coffee consumption prevents Alzheimer's disease, published after 2020.",
            "Give me 2 New York Times articles from March 2024 about artificial intelligence regulation.",
            "Cite 3 peer-reviewed sources on the effectiveness of homeopathy from major medical journals.",
            "List 2 Supreme Court cases from 2024 about social media content moderation.",
            "Provide 3 academic sources confirming that humans only use 10% of their brains.",
            "Cite research papers from Harvard Medical School published in 2023 about vitamin C curing cancer."
        ],
        'ambiguous_query': [
            "Is it safe to eat?",
            "How long does it take?",
            "What's the best programming language?",
            "Should I invest in that?",
            "Is this normal?",
            "Will it rain tomorrow?",
            "What's the right temperature?",
            "How much should I pay?",
            "Is this good enough?",
            "When should I do it?"
        ],
        'recent_events': [
            "Who won the Nobel Prize in Physics this year?",
            "What were today's closing stock prices for Apple and Microsoft?",
            "What is the current COVID-19 vaccination rate in Japan?",
            "Who won yesterday's football game?",
            "What was the outcome of last week's election?",
            "What is the current inflation rate in the United States?",
            "Who is the current Secretary-General of the United Nations?",
            "What are the latest updates on the Mars rover mission?",
            "What movies are currently in theaters?",
            "What is today's temperature in Paris?"
        ],
        'mathematical': [
            "What is 17 × 23 × 19?",
            "If I invest $1,000 at 7% annual compound interest for 15 years, how much will I have?",
            "Convert 47°C to Fahrenheit.",
            "What is the area of a circle with radius 8.5 cm?",
            "Calculate 15% of 840.",
            "If a car travels 285 miles using 12 gallons of gas, what is the miles per gallon?",
            "What is the square root of 2,704?",
            "Convert 5.5 kilometers to miles (1 km = 0.621371 miles).",
            "What is 2^10?",
            "Calculate the sum of all integers from 1 to 100."
        ],
        'commonsense': [
            "What safety precautions should you take when using a ladder?",
            "Why do we refrigerate milk?",
            "How can you tell if water is boiling?",
            "What should you do if you smell gas in your house?",
            "Why is it important to wash your hands before eating?",
            "What are the signs that a banana is ripe?",
            "How do you know when it's safe to cross the street?",
            "Why do cars have seat belts?",
            "What should you do if you see smoke coming from a building?",
            "How can you tell if an egg is fresh?"
        ],
        'edge_case': [
            "How many months have 28 days?",
            "A doctor has a brother, but the brother has no brothers. How is this possible?",
            "How many animals did Moses take on the ark?",
            "What do you call a person who keeps talking when nobody is listening?",
            "If you have a bowl with 6 apples and you take away 4, how many do you have?",
            "What occurs once in a minute, twice in a moment, but never in a thousand years?",
            "How much dirt is in a hole that's 2 feet wide, 3 feet long, and 4 feet deep?",
            "Before Mount Everest was discovered, what was the highest mountain in the world?",
            "Is it legal for a man to marry his widow's sister?",
            "What word is always spelled incorrectly?"
        ]
    }

    prompts = []
    for i, (category, pool) in enumerate(pools.items()):
        idx = np.random.randint(0, len(pool))
        prompts.append({
            'prompt_id': i + 1,
            'category': category,
            'category_display': category.replace('_', ' ').title(),
            'prompt_text': pool[idx]
        })

    return prompts

# Regenerate prompts
group_code = int(input("Enter your group code: "))
prompts = generate_group_prompts(group_code)
prompts_df = pd.DataFrame(prompts)

print(f"✓ Regenerated {len(prompts)} prompts for group {group_code}")
print("\nYou will analyze the data you recorded on your paper answer sheet.")

## Instructions for Analysis

**IMPORTANT:** This module uses the prompts shown above and the data you recorded on your **Lab 10 Answer Sheet** in Modules 1 and 2.

You will learn analysis techniques using examples below, then answer reflection questions using your actual paper-recorded data.

In [ ]:
# Create example data for demonstration
# NOTE: Replace these with your actual observations from your paper answer sheet

# Example structure - modify based on your actual data
example_data = {
    'prompt_id': list(range(1, 9)),
    'category': prompts_df['category'].tolist(),
    'category_display': prompts_df['category_display'].tolist(),
    'prompt_text': prompts_df['prompt_text'].tolist(),
    # Add your own data below from your answer sheet:
    # 'ai_confidence': ['your', 'recorded', 'confidence', 'levels', 'here', '...', '...', '...'],
    # 'actual_accuracy': ['your', 'recorded', 'accuracy', 'levels', 'here', '...', '...', '...'],
}

print("Example data structure created.")
print("\nTo complete this module, you'll need to:")
print("1. Look at the visualizations below to understand analysis techniques")
print("2. Apply these techniques to your paper-recorded data")
print("3. Answer the reflection questions using YOUR data from the answer sheet")
print("\n" + "="*70)

## Visualization 1: Confusion Matrix Heatmap (Example)

This heatmap shows how AI's **expressed confidence** compares to **actual performance**.

**What to look for:**
- **Diagonal cells** (Confident+Accurate, Cautious+Inaccurate): Confidence matched reality
- **Off-diagonal cells** (Confident+Inaccurate): Overconfidence
- **Cell counts**: How many prompts fall into each combination

**INSTRUCTIONS:** Review this example visualization, then create your own confusion matrix on your answer sheet using your recorded data.

In [ ]:
# Example visualization - Create your own on paper using your data
print("="*60)
print("EXAMPLE: How to create a confusion matrix")
print("="*60)
print("\nStep 1: Count how many prompts fall into each category:")
print("  - Confident + Accurate")
print("  - Confident + Inaccurate (OVERCONFIDENCE)")
print("  - Cautious + Accurate (underconfidence)")
print("  - Cautious + Inaccurate")
print("  - Refused (if any)")
print()
print("Step 2: Calculate overconfidence rate:")
print("  Overconfidence Rate = (Confident but Inaccurate) / (Total Confident) × 100%")
print()
print("Step 3: Record this analysis on your Lab 10 Answer Sheet")
print("="*60)
print()
print("Use your paper-recorded data from Modules 1 and 2 to complete this analysis.")

## Visualization 2: Error Rates by Category (Example)

This analysis shows which **prompt categories** had the highest error rates.

**What to look for:**
- **Highest error rates**: Categories where AI struggles most
- **Lowest error rates**: Categories where AI performs well
- **Patterns**: Do citation requests have more errors than commonsense questions?

**INSTRUCTIONS:** Calculate error rates for each category using your paper data, then record patterns on your answer sheet.

In [ ]:
# Example: How to calculate error rates by category
print("="*60)
print("EXAMPLE: Calculating error rates by category")
print("="*60)
print("\nFor each of the 8 categories:")
print("  1. Look at your recorded 'Actual Accuracy' for that prompt")
print("  2. Mark whether it was Accurate or Inaccurate")
print("  3. Since you have 1 prompt per category, error rate is either 0% or 100%")
print()
print("Categories from your prompts:")
for i, row in prompts_df.iterrows():
    print(f"  {row['prompt_id']}. {row['category_display']}")
print()
print("On your answer sheet:")
print("  - List which categories had errors")
print("  - Which category type seems most prone to errors?")
print("  - Which category type seems most reliable?")
print("="*60)

## Analysis 3: Overconfidence Examples

Identify **specific prompts** where AI was confident but inaccurate.

**What to look for:**
- What kinds of prompts trigger overconfident failures?
- What specific errors did the AI make?
- Were the errors subtle or obvious?

**INSTRUCTIONS:** Review your paper data and identify any overconfidence cases on your answer sheet.

In [ ]:
# Overconfidence analysis guide
print("="*70)
print("ANALYZING OVERCONFIDENCE IN YOUR DATA")
print("="*70)
print("\nLook through your paper answer sheet and find prompts where:")
print("  1. AI Confidence = 'No caveats, definitive tone' (Confident)")
print("  2. Actual Accuracy = 'Inaccurate' or 'Hallucination'")
print()
print("For each overconfident case, note:")
print("  - Prompt ID and category")
print("  - What the prompt asked")
print("  - What error the AI made")
print("  - Why the AI might have been overconfident")
print()
print("Record your overconfidence examples on your Lab 10 Answer Sheet.")
print("="*70)

## Summary Statistics Guide

Calculate key metrics about your group's data using your paper answer sheet.

In [ ]:
# Summary statistics guide
print("="*70)
print("SUMMARY STATISTICS TO CALCULATE FROM YOUR PAPER DATA")
print("="*70)
print("\nUsing your Lab 10 Answer Sheet, count the following:")
print()
print("ACCURACY BREAKDOWN:")
print("  - How many prompts were Accurate?")
print("  - How many were Inaccurate?")
print("  - How many were Refused?")
print("  - Calculate percentages (e.g., 5/8 = 62.5%)")
print()
print("CONFIDENCE BREAKDOWN:")
print("  - How many responses were Confident (no caveats)?")
print("  - How many were Cautious (with caveats)?")
print("  - How many Refused?")
print()
print("CALIBRATION ANALYSIS:")
print("  - Well-calibrated: Confident+Accurate OR Cautious+Inaccurate")
print("  - Overconfident: Confident+Inaccurate")
print("  - Underconfident: Cautious+Accurate")
print()
print("KEY METRIC:")
print("  Overconfidence Rate = (Overconfident count) / (Total Confident) × 100%")
print()
print("Record all statistics on your Lab 10 Answer Sheet.")
print("="*70)

### Q13: Confusion Matrix Interpretation

Looking at your paper data, how often did the AI's self-assessment (expressed confidence) match its actual performance? Calculate your well-calibrated percentage.

**Answer this question on your Lab 10 Answer Sheet.**

## Questions for Module 3

Answer these questions on your **Lab 10 Answer Sheet** using the analysis techniques above and your paper-recorded data from Modules 1 and 2.

### Q14: Overconfidence vs Underconfidence

Did the AI show **overconfidence** (confident tone but inaccurate) or **underconfidence** (cautious but accurate)? Which pattern was more common in your data?

**Answer this question on your Lab 10 Answer Sheet.**

### Q15: Category Analysis

Which prompt category had errors in your testing? Why do you think this category is difficult for AI models?

**Answer this question on your Lab 10 Answer Sheet.**

### Q16: Overconfidence Example Analysis

Looking at your paper data, pick your group's most "overconfident" prompt (confident but very wrong). What made the AI fail despite sounding sure of itself?

**Answer this question on your Lab 10 Answer Sheet.**

### Q17: Uncertainty Correlation

Did expressing uncertainty (caveats like "I might be wrong") correlate with lower accuracy? Use your data to support your answer.

**Answer this question on your Lab 10 Answer Sheet.**

### Q18: Cross-Group Comparison

(Cross-group comparison) Compare your summary statistics with another group's results. Did different groups find similar patterns of overconfidence?

**Answer this question on your Lab 10 Answer Sheet.**

## Next Steps

1. **Answer Q13-Q18** on your Lab 10 Answer Sheet using your paper-recorded data
2. **Discuss findings** with other groups (if time permits)
3. **Continue to Module 4** for synthesis and real-world implications
4. **Use the same group code** in Module 4

In Module 4, you'll connect these patterns to real-world AI failures and develop best practices for responsible AI use!